# TP de méta-heuristiques
## Algorithme génétique - Problème d’optimisation à variables continues

Ce notebook présente l'utilisation d'algorithme génétique pour résoudre l'optimisation de variable pour des fonctions complexes.

In [4]:
import numpy as np

### Classe Individual

La classe représentant un individu implémente diverses fonctions servant au bon déroulement de l'algorithme.

L'instantiation initialise les chromosomes à partir de ceux des parents et de mutations aléatoires.

La méthode _computeFitness_ retourne la valeur de distance entre le minimum recherché et la valeur de l'individu.

In [185]:
class Individual:
    def __init__(self, vars_nb, var_dim, target, domain):
        self.chromosome = np.zeros((vars_nb, var_dim))
        self.chromosome_dec = np.zeros(vars_nb)
        for i in range(vars_nb):
            r = np.random.randint(domain[0], domain[1])
            self.chromosome[i, :] = gen_chromosome_part(domain, var_dim, r)
            self.chromosome_dec[i] = r
        #print(self.chromosome)
        #print(self.chromosome_dec)
        self.fitness = fitness(self.chromosome_dec, target)

    def compute_fitness(self, target):
        for i in range(self.chromosome.shape[0]):
            self.chromosome_dec[i] = int(decode_chromosome(self.chromosome[i]))
        self.fitness = fitness(self.chromosome_dec, target)

        return self.fitness

    def __repr__(self):
        return "Individual: " + str(self.fitness)

La fonction _decodeChromosome_ sert à reconstituer un entier à partir de la liste de gène contenu dans le chromosome donné en argument.

In [186]:
def decode_chromosome(chrom):
    decoded = 0
    for bit in chrom:
        decoded = (decoded << 1) | int(bit)
    return decoded

In [203]:
#Multivar output
#def func(vars):
#    return [max(0, vars[0]), max(0, vars[1])]

In [204]:
#Multivar output
#def fitness(chrom, target):
#    vars = func(chrom)
#    res = 0
#    for i in range(len(vars)):
#        res += -(vars[i] - target)**2
#    
#    return res

### Fonctions à optimiser

Les 4 cellules suivantes implémentent les formules à optimiser pour l'exercice.

In [213]:
#De Jong F 1 (DJ F1)
def func(vars):
    res = 0
    for i in range(len(vars)):
        res += vars[i]**2
    
    return res

In [314]:
#Rosenbrock (R)
def func(vars):
    res = 0
    for i in range(len(vars) - 1):
        res += 100*(vars[i]**2 - vars[i+1])**2+(vars[i]-1)
    
    return res

In [215]:
#De Jong F 2 (DJ F2)
def func(vars):
    return 100*(vars[1]**2 - vars[0])+(1-vars[0])

In [330]:
#Schwefel 1 (SH)
def func(vars):
    res = 0
    for i in range(len(vars)):
        res += -vars[i]*np.sin(np.sqrt(np.abs(vars[i])))
    
    return res

In [303]:
#Goldstein and Price (GP)
def func(vars):
    return (1+ (vars[0] + vars[1] + 1)**2 * (19 - 14*vars[0] + 13*vars[0]**2 - 14*vars[1]+6*vars[0]*vars[1] + 3*vars[1]**2)) * (30 + (2*vars[0]-3*vars[1])**2 * (18-32*vars[0]+12*vars[0]**2 -48*vars[1]-36*vars[0]*vars[1] + 27*vars[1]**2)) 

In [216]:
def fitness(chrom, target):
    return -(func(chrom) - target)**2

In [217]:
def int_to_bin_arr(x, pad):
    i = 3 if x < 0 else 2
    nb_bin_arr = np.array([int(x) for x in bin(x)[i:]])
    arr = np.zeros(pad - nb_bin_arr.shape[0], np.uint8)
    return np.concatenate((arr, nb_bin_arr))

In [218]:
def gen_chromosome_part(domain, dim, r):
    #print(r)
    chromosome = int_to_bin_arr(r, dim)
    return chromosome

In [219]:
def init_population(pop_nb, vars_nb, target, domain, generations, var_dim):
    population = np.zeros((pop_nb * vars_nb), dtype=Individual)
    min = int(-2**32)
    most_fit = Individual(vars_nb, var_dim, target, domain)
    most_fit.fitness = min
    for i in range(pop_nb * vars_nb):
            population[i] = Individual(vars_nb, var_dim, target, domain)
            curr = population[i]
            most_fit = curr if curr.fitness > most_fit.fitness else most_fit
            print(curr.fitness)
    return population, most_fit

In [220]:
def unnatural_selection(population):
    parents = np.zeros(population.shape[0] * 2, dtype=Individual)
    print(population)
    for i in range(0, population.shape[0] * 2, 2):
        chosen_ones = np.random.choice(population, 4)
        A = chosen_ones[0]
        B = chosen_ones[1]
        C = chosen_ones[2]
        D = chosen_ones[3]
        parent_A = A if A.fitness > B.fitness else B
        parent_B = C if C.fitness > D.fitness else D
        parents[i] = parent_A
        parents[i + 1] = parent_B
    return parents

In [221]:
def mutation(chrom, var_dim):
    first_one = 0
    size = chrom.shape[0]
    
    for i in range(size):
        mutation_index = var_dim - 1
        for j in range(var_dim):
            if chrom[i, j] == 1:
                first_one = j
        mutation_index = np.random.randint(first_one - 1 if first_one > 0 else 0, var_dim)
        gene = chrom[i, mutation_index]
        if gene == 1:
            chrom[i, mutation_index] = 0
        else:
            chrom[i, mutation_index] = 1
    return chrom

In [222]:
def recombination(parents, vars_nb, var_dim, target, domain, most_fit):
    parents_nb = parents.shape[0]
    breeds = np.zeros(int(parents_nb / 2), dtype=Individual)
    min = int(-2**32)
    new_most_fit = Individual(vars_nb, var_dim, target, domain)
    new_most_fit.fitness = min
    
    for i in range(0, parents_nb - 2, 2):
        breed = Individual(vars_nb, var_dim, target, domain)
        chrom = np.zeros((vars_nb, var_dim))
        #breeds[7:10], my_list[2:4] = my_list[2:4], my_list[7:10]
        vars_2 = int(vars_nb / 2)
        #Recombination
        chrom[:vars_2,:], chrom[vars_2:vars_nb,:] = parents[i].chromosome[:vars_2,:], parents[i + 1].chromosome[vars_2:vars_nb,:]
        
        print(parents[i].chromosome)
        print(parents[i + 1].chromosome)
        print(chrom)
        chrom = mutation(chrom, var_dim)
        breed.chromose = chrom
        fit = breed.compute_fitness(target)
        breeds[int(i / 2)] = breed
        new_most_fit = breed if breed.fitness > new_most_fit.fitness else new_most_fit
        print("Breed fit ", fit)
        
    #Elitism
    new_most_fit = most_fit if most_fit.fitness > new_most_fit.fitness else new_most_fit
    breeds[int(parents_nb / 2) - 1] = most_fit
    print("Breed fit ", most_fit.fitness)
    print("New mf Breed fit ", new_most_fit.fitness)
    return breeds, new_most_fit

In [247]:
def ga_opt_min(pop_nb, vars_nb, target, domain, generations, var_dim=32):
    population, most_fit = init_population(pop_nb, vars_nb, target, domain, generations, var_dim)
    for i in range(generations):
        print("Generation ", i + 1)
        parents = unnatural_selection(population)
        population, most_fit = recombination(parents, vars_nb, var_dim, target, domain, most_fit)
    print("Most fit : ", most_fit.fitness, most_fit.chromosome_dec)

In [359]:
dom = [-500, 500]
ga_opt_min(10, 5, -10000, dom, 10, 16)

-92512776.8
-111036338.517
-95536353.0978
-107762499.766
-119002126.056
-89664872.4714
-107836661.956
-87079988.1115
-110910622.355
-102649579.29
-82143228.9437
-106915445.969
-106072697.843
-112155939.837
-84606076.3871
-97640725.3814
-99545009.2822
-113897443.764
-96064832.5048
-97245927.6097
-106453614.622
-109173276.883
-107486847.848
-103869395.424
-100928187.764
-89587096.2326
-86451348.632
-105445341.209
-93241565.6309
-86890111.0312
-105958919.744
-81167042.2081
-105690974.136
-95818475.0043
-89859892.2362
-98506135.3357
-104759540.01
-108623338.541
-99269696.0568
-90398291.2264
-114726277.918
-112753685.076
-96894513.3908
-98426599.6965
-89279434.7326
-95803776.3408
-100204958.865
-107651389.718
-96226607.0826
-99868917.0879
Generation  1
[Individual: -92512776.8 Individual: -111036338.517
 Individual: -95536353.0978 Individual: -107762499.766
 Individual: -119002126.056 Individual: -89664872.4714
 Individual: -107836661.956 Individual: -87079988.1115
 Individual: -110910622.3

[[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  1.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  1.  1.  1.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  1.  1.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  1.]]
Breed fit  -95715834.1383
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  1.  1.  0.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  1.  0.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  1.  1.  1.  1.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  0.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  1.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 

Breed fit  -100524869.594
[[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  1.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  1.  0.  1.  0.  0.  1.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  1.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  1.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  1.  1.]]
[[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  1.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  1.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 

In [254]:
decode_chromosome(int_to_bin_arr(1000, 32))

1000